<a href="https://colab.research.google.com/github/lucia1970-student/GA-PyTorch/blob/main/NEAT_Impl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install neat-python librosa numpy soundfile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 1.4 MB/s eta 0:00:00


In [ ]:
import neat
import numpy as np
import librosa
import os
import random

# Load dataset (Google Speech Commands or custom dataset)
DATASET_PATH = "path/to/speech_commands"  # Change this to your dataset path
CLASSES = ["yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go"]  # Modify as needed

def extract_features(file_path, max_pad=50):
    """ Extract MFCC features from an audio file """
    y, sr = librosa.load(file_path, sr=16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

    # Padding to ensure equal feature length
    if mfcc.shape[1] < max_pad:
        pad_width = max_pad - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_pad]

    return mfcc.flatten()  # Convert to 1D

# Load dataset and extract features
def load_data():
    X, y = [], []

    for label_idx, label in enumerate(CLASSES):
        folder_path = os.path.join(DATASET_PATH, label)
        for file in os.listdir(folder_path):
            if file.endswith(".wav"):
                file_path = os.path.join(folder_path, file)
                features = extract_features(file_path)
                X.append(features)
                y.append(label_idx)

    return np.array(X), np.array(y)

# Load training data
X, y = load_data()
num_inputs = X.shape[1]
num_outputs = len(CLASSES)

# NEAT Fitness Function
def eval_genomes(genomes, config):
    """ Evaluates genomes using NEAT """
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)

        correct = 0
        for i in range(len(X)):
            output = net.activate(X[i])
            predicted_label = np.argmax(output)
            if predicted_label == y[i]:
                correct += 1

        genome.fitness = correct / len(X)  # Accuracy as fitness

# NEAT Configuration
def run_neat():
    """ Runs NEAT algorithm for evolving neural networks """
    config_path = "config-feedforward"  # Create a NEAT config file
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)

    population = neat.Population(config)
    population.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    population.add_reporter(stats)

    winner = population.run(eval_genomes, 50)  # Run for 50 generations
    return winner

# Run Neuroevolution
best_network = run_neat()

# Test the evolved network
def test_network(network):
    correct = 0
    for i in range(len(X)):
        output = network.activate(X[i])
        predicted_label = np.argmax(output)
        if predicted_label == y[i]:
            correct += 1
    print(f"Final Accuracy: {correct / len(X) * 100:.2f}%")

winner_net = neat.nn.FeedForwardNetwork.create(best_network, neat.Config(
    neat.DefaultGenome, neat.DefaultReproduction,
    neat.DefaultSpeciesSet, neat.DefaultStagnation, "config-feedforward"
))

test_network(winner_net)

FileNotFoundError: [Errno 2] No such file or directory: 'path/to/speech_commands/yes'